In [52]:
'''
CSAAW Hackathon

'''

import numpy as np
from numpy import genfromtxt
from sklearn.metrics.pairwise import euclidean_distances
import sklearn.metrics.pairwise as Kern
from sklearn import preprocessing 
import glob
import os

def getSigma(X,sigmaType):
    [n,d] = X.shape
    
    shat = np.var(X)
    if sigmaType == 'Scott':
        sigma = np.mean(n ** (-1.0/(d+4) * shat))
    elif sigmaType == 'Silverman':
        sigma = np.mean(n ** (-1/(d+4))*shat*(4.0/(d+2.0))**(1.0/(d+4.0)))
    
    return sigma

def getSimilarityMatrix(NormalizationFlag,SimilarityFlag):
    dataset_list = []
    cwd = os.getcwd()
    for files in glob.glob(cwd + "/Site_specific_data/*.csv"):
        dataset_list.append(files)
    
    N = len(dataset_list)
    dataset = genfromtxt(dataset_list[1], delimiter=',')
    datasetTotal = np.zeros([1,dataset.shape[1]-1])
    for site in range(0,N):
    
        dataset_file = dataset_list[site]
        dataset = genfromtxt(dataset_file, delimiter=',')
    
        if len(dataset.shape) == 2:
            datasetX = dataset[:,:-1]
            datasetID = dataset[:,-1]
        else: 
            dataset = dataset[np.newaxis,:]
            datasetX = dataset[:,:-1]
            datasetID = dataset[:,-1]
    
        if NormalizationFlag == 0:
            dataset_normalized = preprocessing.normalize(datasetX, norm='l2')
        elif NormalizationFlag == 1:
            dataset_normalized = preprocessing.scale(datasetX)
        elif NormalizationFlag == 2:
            min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
            dataset_normalized = min_max_scaler.fit_transform(datasetX)
        
        dataset_site = 'dataset' + str(site)
        exec (dataset_site + " = dataset_normalized")
    
        datasetTotal = np.concatenate((datasetTotal,dataset_normalized),axis = 0)
    
    datasetTotal = datasetTotal[1:,:]
    bw_prob = getSigma(datasetTotal,'Scott')
    print bw_prob
    Sim = np.zeros([N,N])
    for ii in range(0,N):
        dataset_ii = 'dataset' + str(ii)
        X_ii = eval(dataset_ii)
        for jj in range(0,N):
            dataset_jj = 'dataset' + str(jj)
            X_jj = eval(dataset_jj)
            if SimilarityFlag == 0:
                SimWithinSite = euclidean_distances(X_ii,X_jj) 
                Sim[ii,jj] = np.linalg.norm(SimWithinSite,ord = 'fro')/(float(M.shape[0])*float(M.shape[1]))
            elif SimilarityFlag == 1:
                SimWithinSite = Kern.rbf_kernel(X_ii,X_jj,bw_prob)
                Sim[ii,jj] = np.mean(SimWithinSite)

    if SimilarityFlag == 2:
        SimFinal = np.zeros([N,N])
        for ii in range(0,N):
            dataset_ii = 'dataset' + str(ii)
            for jj in range(0,N):
                dataset_jj = 'dataset' + str(jj)
                if SimilarityFlag == 0:
                    SimFinal[ii,jj] = Sim[ii,jj]
                elif SimilarityFlag == 1:
                    expVar = Sim[i,i] + Sim[j,j] - 2*Sim[i,j]
                    SimFinal[ii,jj] = np.exp(-sim*bwFinal)
        Sim = np.copy(SimFinal)
    return Sim, datasetTotal


def getSimilarityMatrixEra(NormalizationFlag,SimilarityFlag):
    dataset_list = []
    cwd = os.getcwd()
    for files in glob.glob(cwd + "/Site_specific_data_with_ERA/*.csv"):
        dataset_list.append(files)
    
    N = len(dataset_list)
    dataset = genfromtxt(dataset_list[1], delimiter=',')
    datasetTotal = np.zeros([1,dataset.shape[1]-2])
    for site in range(0,N):
    
        dataset_file = dataset_list[site]
        dataset = genfromtxt(dataset_file, delimiter=',')
    
        if len(dataset.shape) == 2:
            datasetX = dataset[:,:-2]
            datasetERA = dataset[:,-1]
            datasetID = dataset[:,-2]
        else: 
            dataset = dataset[np.newaxis,:]
            datasetX = dataset[:,:-2]
            datasetERA = dataset[:,-1]
            datasetID = dataset[:,-2]
    
        if NormalizationFlag == 0:
            dataset_normalized = preprocessing.normalize(datasetX, norm='l2')
        elif NormalizationFlag == 1:
            dataset_normalized = preprocessing.scale(datasetX)
        elif NormalizationFlag == 2:
            min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
            dataset_normalized = min_max_scaler.fit_transform(datasetX)
        
        dataset_site = 'dataset' + str(site)
        exec (dataset_site + " = dataset_normalized")
        
        dataset_site_era = 'dataset_era' + str(site)
        exec (dataset_site_era + " = datasetERA" )
    
        datasetTotal = np.concatenate((datasetTotal,dataset_normalized),axis = 0)
    
    datasetTotal = datasetTotal[1:,:]
    bw_prob = getSigma(datasetTotal,'Scott')
    print bw_prob
    Sim = np.zeros([N,N])
    for ii in range(0,N):
        dataset_ii = 'dataset' + str(ii)
        X_iiEra = eval('dataset_era' + str(ii))
        X_ii = eval(dataset_ii)
        for jj in range(0,N):
            dataset_jj = 'dataset' + str(jj)
            X_jjEra = eval('dataset_era' + str(jj))
            X_jj = eval(dataset_jj)
            if SimilarityFlag == 0:
                SimWithinSite = euclidean_distances(X_ii,X_jj) 
                Sim[ii,jj] = np.linalg.norm(SimWithinSite,ord = 'fro')/(float(M.shape[0])*float(M.shape[1]))
            elif SimilarityFlag == 1:
                SimWithinSite = Kern.rbf_kernel(X_ii,X_jj,bw_prob)
                EraSim = np.zeros([X_ii.shape[0],X_jj.shape[0]])
                for pp in range(0,X_ii.shape[0]):
                    for qq in range(0,X_jj.shape[0]):
                        EraSim[pp,qq] = X_iiEra[pp] - X_jjEra[qq]
                
                
                Sim[ii,jj] = np.mean(SimWithinSite*EraSim)
    return Sim
    


In [53]:
from sklearn import metrics
from sklearn import cluster
import matplotlib.pyplot as plt

thres=0.04
matrix_type='dist'

NormalizationFlag = 1 # 0 is normalizing by norm, 1 is scale it, 2 is make it in between [0,1]
SimilarityFlag = 1 #0 if euclidean DISTANCE (note distance and similarity), 1 if RKHS (mean embedding) with linear kernel, 2 if RKHS with Gaussian kernel
Sim, datasetTotal = getSimilarityMatrix(NormalizationFlag,SimilarityFlag)

SimEra = getSimilarityMatrixEra(NormalizationFlag,SimilarityFlag)

'''
#n = 100#datasetTotal.shape[0]
#inertia = np.zeros([n])
#for i in range(0,n):
#    centroids,labels,inertia[i] = cluster.k_means(datasetTotal,n_clusters=31)
#plt.plot(range(0,n), inertia)
#plt.ylabel('Distances')
#plt.show()
'''

centroids,labels,inertia = cluster.k_means(datasetTotal,n_clusters=5)


0.803579418463
0.807434765877


In [54]:
print SimEra

[[  0.00000000e+00   8.31534630e-02   3.86000943e-03   1.97482339e-02
    7.64732477e-03   6.72402596e-04   4.45049917e-03   1.47494106e-02
    8.02561233e-02   8.63215742e-03   1.48391567e-02   7.41151807e-03
    1.38241081e-03   8.02824467e-05   7.11968897e-02   7.00654608e-09
    1.31320554e-04   7.67657485e-04   1.50543374e-03   3.46294183e-03
    1.18004510e-03   9.71493877e-07   5.78651941e-03   2.49260838e-03
    7.16068508e-03]
 [ -8.31534630e-02   0.00000000e+00  -2.62606711e-04   1.00688175e-03
    1.37176756e-04   2.95665779e-04   4.51965906e-03  -1.93684274e-05
   -1.11423914e-03   4.95693421e-04   5.71820633e-04   3.41411106e-04
   -5.92379913e-05   2.06036103e-06   1.07938957e-03   2.54027268e-08
    2.85731310e-05   5.40998961e-04   6.51136863e-05  -5.12846111e-04
    7.58584923e-05  -1.41488696e-04   3.56944228e-04   9.63995240e-05
    9.69817464e-05]
 [ -3.86000943e-03   2.62606711e-04   0.00000000e+00   2.74457265e-03
    4.03929423e-03   2.27137253e-04   1.29178555e-

In [ ]:


thres=0.04
matrix_type='dist'

NormalizationFlag = 1 # 0 is normalizing by norm, 1 is scale it, 2 is make it in between [0,1]
SimilarityFlag = 1 #0 if euclidean DISTANCE (note distance and similarity), 1 if RKHS (mean embedding) with linear kernel, 2 if RKHS with Gaussian kernel
Sim = getSimilarityMatrix(NormalizationFlag,SimilarityFlag)


In [38]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


0.803579418463
[[  2.50000005e-01   2.64683179e-05   1.06908283e-05   6.46980913e-07
    1.72651630e-09   4.47441993e-05   2.63184488e-04   9.55082263e-13
    2.33830055e-04   6.00182413e-07   2.30477169e-11   4.02315279e-05
    2.79093361e-08   4.98238143e-04   6.19558063e-06   4.00245570e-07
    9.99688487e-07   3.75153241e-06   1.06075954e-06   1.13003806e-05
    6.08664063e-05   1.25871027e-06   1.83121214e-06   3.40987437e-07
    6.71160677e-06   1.09377955e-05   7.19291589e-07   8.21039443e-07
    2.95873012e-05   1.29429455e-08   3.30149590e-06]
 [  2.64683179e-05   2.29990924e-02   1.22044471e-05   8.54450946e-06
    1.64187619e-06   1.71503949e-04   2.76869312e-04   1.13392265e-09
    3.71470747e-04   2.82361851e-06   1.29056551e-08   4.01528299e-04
    3.66954337e-08   3.76304362e-03   1.67054571e-05   8.72327111e-06
    7.85022978e-06   3.69567845e-06   1.21307025e-05   1.91765346e-05
    7.63841850e-05   1.81041106e-05   2.02862637e-05   6.13677294e-07
    3.89720004e-05   

ImportError: No module named py

In [38]:
import numpy as np
import scipy as sp
import sklearn.metrics.pairwise as Kern

X = np.random.rand(4,2)
X1 = np.random.rand(4,2)
A = Kern.rbf_kernel(X,X1,0.05)
                    
print X*X1

[[ 0.46691177  0.68346885]
 [ 0.46024211  0.39136154]
 [ 0.24173493  0.21591235]
 [ 0.55547421  0.23669852]]
